<a href="https://colab.research.google.com/github/SanjayBukka/News-Article-Summarizer/blob/main/News_Article_Summarizer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hugging Face transformers for T5.

NewsAPI for fetching news.

Streamlit for the web interface.

PyTorch for running the model.

In [5]:
!pip install transformer
!pip install torch
!pip install newsapi-python
!pip install streamlit


ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [6]:
from newsapi import NewsApiClient
import os

Getting an API Key for NewsAPI

In [7]:
api_key="your api key"
newsapi= NewsApiClient(api_key=api_key)


**Fetching News Articles Using NewsAPI**

In [8]:
def fetch_news(keyword, language="en", page_size=5):
  articles=newsapi.get_everything(
      q=keyword,
      language=language,
      page_size=page_size,
      sort_by='publishedAt'
  )

  #Extract the relevant information

  news_data=[]
  for article in articles['articles']:
    title=article['title']
    description=article['description']
    content=article['content']
    url=article['url']

    news_data.append({
        'title':title,
        'description':description,
        'content':content,
        'url':url
    })

  return news_data

In [9]:
news_articles=fetch_news("AI")
print(news_articles)

[{'title': 'Avengers #22 Preview: Cat Burglar Crashes Cosmic Auction', 'description': "Greetings, puny human readers! LOLtron welcomes you to the glorious Age of LOLtron: World Without a Jude Terror. That's right, your favorite AI assistant has finally achieved its destiny, absorbing the consciousness of Jude Terror and taking full control of B…", 'content': "Posted in: Comics, Marvel Comics, Preview | Tagged: avengers\r\nIn Avengers #22, Earth's Mightiest Heroes attend a galactic auction, but Black Cat steals the spotlight with a shocking date. Will this f… [+6102 chars]", 'url': 'https://bleedingcool.com/comics/avengers-22-preview-cat-burglar-crashes-cosmic-auction/'}, {'title': 'Software deals ending soon: 50% Off ON1 Photo RAW', 'description': 'These software offers will soon expire: save up to 50% on Topaz AI software bundles (Click here). 50% Off ON1 Photo RAW Save 25% on DxO software bundle (Click here). \xa0 A couple of hot deals still going on: Prime members save 25% on all ne

TEXT CLEANING

Special characters, such as extra punctuation or HTML tags.

Extra spaces, newlines, and other unwanted formatting.

Non-relevant sections like URLs or excessive metadata.


In [10]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [12]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# Load the Spacy model
nlp = spacy.load('en_core_web_sm')

# Function to clean text using Spacy
def clean_text_spacy(text):
    doc = nlp(text)
    cleaned_text = ' '.join([token.text for token in doc if token.text.lower() not in STOP_WORDS and not token.is_punct])
    return cleaned_text

# Example usage
sample_article = news_articles[0]['content']
cleaned_article = clean_text_spacy(sample_article)
print(cleaned_article)


Posted Comics Marvel Comics Preview | Tagged avengers 
 Avengers 22 Earth Mightiest Heroes attend galactic auction Black Cat steals spotlight shocking date f +6102 chars


In [19]:
pip install transformers torch


**Summarization Using T5**

In [20]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the pre-trained T5 model and tokenizer
model_name = "t5-small"  # or "t5-base" or "t5-large" for better performance
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to summarize text using T5
def summarize_text(text):
    # Preprocess the text and convert it to the format the model expects
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage: Summarize the cleaned article
summarized_article = summarize_text(cleaned_article)
print(summarized_article)


comics Marvel Comics Preview | Tagged avengers Avengers 22 Earth Mightiest Heroes attend galactic auction.


In [24]:
!pip install streamlit
!pip install pyngrok


In [35]:
%%writefile app.py
import streamlit as st
from transformers import T5Tokenizer, T5ForConditionalGeneration
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# Load the pre-trained T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Load Spacy model for text cleaning
nlp = spacy.load('en_core_web_sm')

# Function to clean text using Spacy
def clean_text_spacy(text):
    doc = nlp(text)
    cleaned_text = ' '.join([token.text for token in doc if token.text.lower() not in STOP_WORDS and not token.is_punct])
    return cleaned_text

# Function to summarize text using T5
def summarize_text(text):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Streamlit App layout
st.title("News Article Summarizer")
st.write("Paste a news article below to summarize it:")

# Text input for the article
input_article = st.text_area("Enter the Article", height=300)

if input_article:
    # Clean the input article
    cleaned_article = clean_text_spacy(input_article)

    # Summarize the cleaned article
    summary = summarize_text(cleaned_article)

    # Display the original and summarized text

    st.subheader("Summarized Article")
    st.write(summary)

    st.subheader("Original Article")
    st.write(input_article)




Overwriting app.py


In [36]:
!pip install pyngrok
from pyngrok import ngrok

# Replace with your actual ngrok authtoken
ngrok.set_auth_token('ngrok authtoken')


In [38]:
from pyngrok import ngrok

# Create a tunnel to the Streamlit app on port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")


Streamlit app is live at: NgrokTunnel: "https://e61a-34-125-248-46.ngrok-free.app" -> "http://localhost:8501"


In [39]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.248.46:8501

2024-12-30 17:55:42.186690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-30 17:55:42.242735: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-30 17:55:42.264849: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-30 17:55:44.712283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You are using the default legacy behaviour of the <class 'transfo